# Venture Funding with Deep Learning

A hypothetical venture capital firm Alphabet Soup has provided a CSV file of over 34,000 cases of organizational funding and the outcomes. The request for analysis is to attempt at making a neural networks which can predict whether an approved application for funding will result in a successful business.

First the data will be prepared for analysis. Then several models of neural networks will be run on the data to see what rate of successful prediction can be achieved. The accuracies of the various models will be displayed and compared. The neural network models will be saved in HDF5 format.

In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

---

## Prepare the data to be used on a neural network model

Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define the features and target variables.  


In [2]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv(Path("./Resources/applicants_data.csv"))

# Review the DataFrame
display(applicant_data_df)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Review the amount of successful startups in the dataset
print(applicant_data_df["IS_SUCCESSFUL"].sum())

18261


In [4]:
# Review the proportion of successful startups in the dataset
print(applicant_data_df["IS_SUCCESSFUL"].sum()/ applicant_data_df.shape[0])

0.5324061926003674


In [5]:
# Review the data types associated with the columns
display(applicant_data_df.dtypes)

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

### Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [6]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=["EIN", "NAME"])

# Review the DataFrame
display(applicant_data_df)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


### Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [7]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes=="object"].index)

# Display the categorical variables list
display(categorical_variables)

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [9]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])

In [10]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables))

# Review the DataFrame
display(encoded_df)

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

In [11]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
preprocessed_df = pd.concat([applicant_data_df.drop(columns=categorical_variables), encoded_df], axis=1)

# Review the Dataframe
display(preprocessed_df)

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset is defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns define the features dataset.

In [12]:
# Define the target set y using the IS_SUCCESSFUL column
y = preprocessed_df["IS_SUCCESSFUL"]

# Display a sample of y
display(y)

0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

In [13]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = preprocessed_df.drop(columns=["IS_SUCCESSFUL"])

# Review the features DataFrame
display(X)

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Split the features and target sets into training and testing datasets.


In [14]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Use scikit-learn's `StandardScaler` to scale the features data.

In [15]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

In [16]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
print(number_input_features)


116


In [17]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [18]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = (number_input_features + number_output_neurons)//2

# Review the number hidden nodes in the first layer
display(hidden_nodes_layer1)

58

In [19]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = (hidden_nodes_layer1 + number_output_neurons)//2

# Review the number hidden nodes in the second layer
display(hidden_nodes_layer2)

29

In [20]:
# Create the Sequential model instance
nn = Sequential()

In [21]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [22]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [23]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=1, activation="sigmoid"))

In [24]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


### Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [25]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 556us/step - loss: 0.5763 - accuracy: 0.7215
Epoch 2/50
804/804 [==============================] - 0s 560us/step - loss: 0.5533 - accuracy: 0.7294
Epoch 3/50
804/804 [==============================] - 0s 562us/step - loss: 0.5498 - accuracy: 0.7306
Epoch 4/50
804/804 [==============================] - 0s 562us/step - loss: 0.5480 - accuracy: 0.7311
Epoch 5/50
804/804 [==============================] - 0s 560us/step - loss: 0.5463 - accuracy: 0.7317
Epoch 6/50
804/804 [==============================] - 0s 561us/step - loss: 0.5450 - accuracy: 0.7324
Epoch 7/50
804/804 [==============================] - 0s 560us/step - loss: 0.5441 - accuracy: 0.7346
Epoch 8/50
804/804 [==============================] - 0s 559us/step - loss: 0.5434 - accuracy: 0.7332
Epoch 9/50
804/804 [==============================] - 0s 558us/step - loss: 0.5429 - accuracy: 0.7341
Epoch 10/50
804/804 [==============================] - 0s 570us/step - loss: 0.542

### Evaluate the model using the test data to determine the model’s loss and accuracy.

In [27]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5533 - accuracy: 0.7300 - 157ms/epoch - 587us/step
Loss: 0.5533246994018555, Accuracy: 0.7300291657447815


### Save and export the first model to an HDF5 file. Name the file `AlphabetSoup.h5`. 

In [ ]:
# Set the model's file path
file_path = Path("./AlphabetSoup.h5")

# Export your model to a HDF5 file
nn.save(file_path)

---

## Optimize the neural network model


### Several attempts will be made to optimize the neural network model, to see if increased accuracy can be achieved.

Here, four attempts are made:

- The first alternative model will be identical to the first one, except with some different preprocessing of the data. Two columns which have all the same value are removed. The column of the income amount of the organization is changed from a range to the value at the upper end of that range. The income amount and ask amounts are first scaled with a natural log function before StandardScaler is applied.

- The second alternative model will also use the same alternative preprocessing and neural network structure as the first, but use the Nadam optimization function.

- The third alternative model will ukse the same alternative preprocessing, the Nadam optimization function, but use four layers of neurons.

- The fourth alternative model will attempt to see if a higher rate of accuracy can be achieved with only considering data for funding ask amounts of under 1 million dollars. It will use the same preprocessing as the other alternative models and the Nadam optimization function, and with two layers of neurons.

### Alternative Preprocessing 1:

In [ ]:
# Preprocessing Alt-1:

In [ ]:
# remove STATUS and SPECIAL_CONSIDERATIONS columns, as they appear to have all the same value in them.
applicant_data_df_a1 = applicant_data_df.drop(columns=["STATUS", "SPECIAL_CONSIDERATIONS"])

In [ ]:
# review new dataframe
display(applicant_data_df_a1)

In [ ]:
# retrieve max ASK_AMT
applicant_data_df_a1["ASK_AMT"].max()

In [ ]:
# apply natural log scale to ASK_AMT
applicant_data_df_a1["ASK_AMT"]=np.log(applicant_data_df_a1["ASK_AMT"])

In [ ]:
# review modified dataframe
display(applicant_data_df_a1)

In [ ]:
# find all categories of INCOME_AMT
applicant_data_df_a1["INCOME_AMT"].value_counts()

In [ ]:
# relabel each income group by the max value it can represent. for 50M+, will use 10 billion, as the highest value asked for in the data set is in the 8 billion range. Used 1 for 0 so log function will work and return 0.
applicant_data_df_a1["INCOME_AMT"] = applicant_data_df_a1["INCOME_AMT"].replace(["0", "25000-99999", "100000-499999", "1M-5M", "1-9999", "10000-24999", "10M-50M", "5M-10M", "50M+"], [1, 100000, 500000, 5000000, 10000, 25000, 50000000, 10000000, 10000000000])

In [ ]:
# review modified dataframe
display(applicant_data_df_a1)

In [ ]:
# apply natural log to income amounts
applicant_data_df_a1["INCOME_AMT"]=np.log(applicant_data_df_a1["INCOME_AMT"])

In [ ]:
# review modified dataframe
display(applicant_data_df_a1)

In [29]:
# Recapitulation of preprocessing steps taken above, onehot encoding and standard scaler, but for the data with irrelevant columns removed and natural log for income and ask amounts.

In [ ]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df_a1.dtypes[applicant_data_df_a1.dtypes=="object"].index)

# Display the categorical variables list
display(categorical_variables)

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [ ]:
# Encode the categorcal variables using OneHotEncoder
encoded_data_a1 = enc.fit_transform(applicant_data_df_a1[categorical_variables])

In [ ]:
# Create a DataFrame with the encoded variables
encoded_df_a1 = pd.DataFrame(
    encoded_data_a1,
    columns = enc.get_feature_names(categorical_variables))

# Review the DataFrame
display(encoded_df_a1)

In [ ]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
preprocessed_df_a1 = pd.concat([applicant_data_df_a1.drop(columns=categorical_variables), encoded_df_a1], axis=1)

# Review the Dataframe
display(preprocessed_df_a1)

In [ ]:
# Define the target set y using the IS_SUCCESSFUL column
y = preprocessed_df_a1["IS_SUCCESSFUL"]

# Display a sample of y
display(y)

In [ ]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = preprocessed_df_a1.drop(columns=["IS_SUCCESSFUL"])

# Review the features DataFrame
display(X)

In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Alternative Model 1

This model uses the same neural network structure as the first one, but uses alternatively preprocessed data. Several irrelevant columns were removed, income amounts ranges were replaced with representative values, and income and ask amounts were applied natural log functions before scaling.

In [ ]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

In [ ]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

In [ ]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (number_input_features + number_output_neurons_A1)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

In [ ]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 = (hidden_nodes_layer1_A1 + number_output_neurons)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer2_A1

In [ ]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [ ]:
# First hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))          
          
# Output layer
nn_A1.add(Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_A1.summary()

In [ ]:
# Compile the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50)

#### Alternative Model 2

In [ ]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

In [ ]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [ ]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = (number_input_features + number_output_neurons_A2)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

In [ ]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 = (hidden_nodes_layer1_A2 + number_output_neurons_A2)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer2_A2

In [ ]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [ ]:
# First hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))          
          
# Output layer
nn_A2.add(Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_A2.summary()

In [ ]:
# Compile the model
nn_A2.compile(loss="binary_crossentropy", optimizer="Nadam", metrics=["accuracy"])


In [ ]:
# Fit the model
fit_model_A2 = nn_A2.fit(X_train_scaled, y_train, epochs=50)


#### Alternative Model 3

In [ ]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

In [ ]:
# Define the number of neurons in the output layer
number_output_neurons_A3 = 1

In [ ]:
# Define the number of hidden nodes for the hidden layers
hidden_nodes_layer1_A3 = 26
hidden_nodes_layer2_A3 = 16
hidden_nodes_layer3_A3 = 6
hidden_nodes_layer4_A3 = 3

In [ ]:
# Create the Sequential model instance
nn_A3 = Sequential()

In [ ]:
# First hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer1_A3, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer2_A3, activation="relu"))          
   
# Third hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer3_A3, activation="relu"))          
          
# Fourth hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer4_A3, activation="relu"))               
    
# Output layer
nn_A3.add(Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_A3.summary()

In [ ]:
# Compile the model
nn_A3.compile(loss="binary_crossentropy", optimizer="Nadam", metrics=["accuracy"])


In [ ]:
# Fit the model
fit_model_A3 = nn_A3.fit(X_train_scaled, y_train, epochs=50)


### Alternative Preprocessing 2:

In [ ]:
# Preprocessing Alt-2:

In [ ]:
# remove STATUS and SPECIAL_CONSIDERATIONS columns, as they appear to have all the same value in them.
applicant_data_df_a2 = applicant_data_df.drop(columns=["STATUS", "SPECIAL_CONSIDERATIONS"])

In [ ]:
applicant_data_df_a2

In [ ]:
# drop all rows with funding requests over $1,000,000
small_funding_df = applicant_data_df_a2.loc[(applicant_data_df_a2["ASK_AMT"] <= 200000)]

In [ ]:
small_funding_df

In [ ]:
# apply natural log scale to ASK_AMT
small_funding_df["ASK_AMT"]=np.log(small_funding_df["ASK_AMT"])

In [ ]:
display(small_funding_df)

In [ ]:
# relabel each income group by the max value it can represent. for 50M+, will use 10 billion, as the highest value asked for in the data set is in the 8 billion range. Used 1 for 0 so log function will work and return 0.
small_funding_df["INCOME_AMT"] = small_funding_df["INCOME_AMT"].replace(["0", "25000-99999", "100000-499999", "1M-5M", "1-9999", "10000-24999", "10M-50M", "5M-10M", "50M+"], [1, 100000, 500000, 5000000, 10000, 25000, 50000000, 10000000, 10000000000])

In [ ]:
display(small_funding_df)

In [ ]:
# apply natural log to income amounts
small_funding_df["INCOME_AMT"]=np.log(small_funding_df["INCOME_AMT"])

In [ ]:
display(small_funding_df)

In [ ]:
# reset the index of the small funding dataframe
small_funding_df = small_funding_df.reset_index()

In [ ]:
# recapitulation of preprocessing steps taken above, onehot encoding and standard scaler, but for the data with irrelevant columns removed and natural log for income and ask amounts.

In [ ]:
# Create a list of categorical variables 
categorical_variables = list(small_funding_df.dtypes[small_funding_df.dtypes=="object"].index)

# Display the categorical variables list
categorical_variables


In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [ ]:
# Encode the categorcal variables using OneHotEncoder
encoded_data_a2 = enc.fit_transform(small_funding_df[categorical_variables])


In [ ]:
# Create a DataFrame with the encoded variables
encoded_df_a2 = pd.DataFrame(
    encoded_data_a2,
    columns = enc.get_feature_names(categorical_variables))

# Review the DataFrame
encoded_df_a2

### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [ ]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
preprocessed_df_a2 = pd.concat([small_funding_df.drop(columns=categorical_variables), encoded_df_a2], axis=1)

# Review the Dataframe
preprocessed_df_a2


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [ ]:
# Define the target set y using the IS_SUCCESSFUL column
y = preprocessed_df_a2["IS_SUCCESSFUL"]

# Display a sample of y
display(y)


In [ ]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = preprocessed_df_a2.drop(columns=["IS_SUCCESSFUL"])

# Review the features DataFrame
display(X)

### Step 6: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, random_state=1)


### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X2_scaler = scaler.fit(X2_train)

# Fit the scaler to the features training dataset
X2_train_scaled = scaler.transform(X2_train)
X2_test_scaled = scaler.transform(X2_test)


In [ ]:
display(pd.DataFrame(X2_train_scaled))

In [ ]:
# Define the the number of inputs (features) to the model
number_input_features = len(X2_train.iloc[0])

# Review the number of features
number_input_features

In [ ]:
# Define the number of neurons in the output layer
number_output_neurons_A4 = 1

In [ ]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A4 = (number_input_features + number_output_neurons_A4)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A4

In [ ]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A4 = (hidden_nodes_layer1_A4 + number_output_neurons_A4)//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer2_A4

In [ ]:
# Create the Sequential model instance
nn_A4 = Sequential()

In [ ]:
# First hidden layer
nn_A4.add(Dense(units=hidden_nodes_layer1_A2, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_A4.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))          
          
# Output layer
nn_A4.add(Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_A4.summary()

In [ ]:
# Compile the model
nn_A4.compile(loss="binary_crossentropy", optimizer="Nadam", metrics=["accuracy"])


In [ ]:
# Fit the model
fit_model_A4 = nn_A4.fit(X2_train_scaled, y2_train, epochs=50)


### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [ ]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = # YOUR CODE HERE

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy =# YOUR CODE HERE

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = # YOUR CODE HERE

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = # YOUR CODE HERE

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = # YOUR CODE HERE

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### Step 3: Save each of your alternative models as an HDF5 file.


In [ ]:
# Set the file path for the first alternative model
file_path = # YOUR CODE HERE

# Export your model to a HDF5 file
# YOUR CODE HERE


In [ ]:
# Set the file path for the second alternative model
file_path = # YOUR CODE HERE

# Export your model to a HDF5 file
# YOUR CODE HERE
